In [357]:
using JSON, SDDP
data = JSON.parsefile("model.parameters.json")
results = SDDP.load("model/Powder.results");

In [358]:
using Plots
upscale = 1 #8x upscaling in resolution
fntsm = Plots.font("times", 10.0pt)
fntlg = Plots.font("times", 12.0pt)
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm)#,left_margin=10mm,bottom_margin=7.5mm)
default(size=(800,600),top_margin=0mm, right_margin=0mm) #Plot canvas size
gr()

Plots.GRBackend()

In [359]:
plot(size=(750,500), left_margin=5mm, bottom_margin=5mm)
milking_requirements = data["energy_for_pregnancy"] + data["energy_for_bcs_milking"] + data["energy_for_maintenance"]
dry_requirements = data["energy_for_pregnancy"] + data["energy_for_bcs_dry"] + data["energy_for_maintenance"]

plot!(milking_requirements, label="Total: Milking",          linewidth=3, color="#00467F")
plot!(dry_requirements, label="Total: Dry",                  linewidth=3, color="#00467F", linestyle=:dot)
plot!(data["energy_for_bcs_milking"],label="BCS - Milking",  linewidth=3, color="#e65100")
plot!(data["energy_for_bcs_dry"],label="BCS - Dry",          linewidth=3, color="#e65100", linestyle=:dot)
hline!([data["energy_for_maintenance"]],label="Maintenance", linewidth=3, color="#009AC7", linestyle=:solid)
plot!(data["energy_for_pregnancy"], label="Pregnancy",       linewidth=3, color="#009AC7", linestyle=:dot)

plot!(ylims=(-100,900), xlabel="Weeks since calving", ylabel="Energy Requirement (MJ/Week)", legend=:topleft)
savefig("energy.pdf")

In [365]:
function plotpowder(results, key, ylims, ylabel,title)
    maxidx = indmax([results[i][:markov][35] for i in 1:length(results)])
    minidx = indmin([results[i][:markov][35] for i in 1:length(results)])
    plot(
        hcat([results[i][key] for i in 1:length(results) if results[i][:markov][40] == 2]...),
        color="gray", alpha=0.03, linewidth=1
    )
    plot!(
        hcat([results[i][key] for i in 1:length(results) if results[i][:markov][40] == 1]...),
        color="#00467F", alpha=0.03, linewidth=1
    )
    
    plot!(
        hcat([results[i][key] for i in 1:length(results) if results[i][:markov][40] == 3]...),
        color="#e65100", alpha=0.03, linewidth=1
    )

    plot!(results[maxidx][key], color="#e65100", linewidth=3)
    plot!(results[minidx][key], color="#00467F", linewidth=3)

    plot!(legend=false, ylims=ylims, xlabel="Weeks since calving\n", ylabel=ylabel,title=title)
end

function plotprice(results, title)
    maxidx = indmax([results[i][:markov][35] for i in 1:length(results)])
    minidx = indmin([results[i][:markov][35] for i in 1:length(results)])
    prices = Vector{Float64}[]
    for t in 1:24
        push!(prices, [6.0])
    end
    for t in 25:51
        push!(prices, [5.0, 6.0, 7.0])
    end
    push!(prices, [4.0, 5.0, 6.0, 7.0, 8.0])

    function toprices(markov)
       [prices[t][i] for (t,i) in enumerate(markov)] 
    end
    plot(
        hcat([toprices(results[i][:markov]) for i in 1:length(results) if results[i][:markov][40] == 2]...),
        color="gray", linewidth=1, alpha=0.03
    )
    plot!(
        hcat([toprices(results[i][:markov]) for i in 1:length(results) if results[i][:markov][40] == 1]...),
        color="#00467F", linewidth=1, alpha=0.03
    )

    plot!(
        hcat([toprices(results[i][:markov]) for i in 1:length(results) if results[i][:markov][40] == 3]...),
        color="#e65100", linewidth=1, alpha=0.03
    )
    plot!(toprices(results[maxidx][:markov]), color="#e65100", linewidth=3)
    plot!(toprices(results[minidx][:markov]), color="#00467F", linewidth=3)
    plot!(legend=false, xlabel="Weeks since calving\n", ylabel="Forecast Price\n(\$/kg)",title=title)
end
                                                                                
plot(
    plotprice(results, "(a)"),
    plotpowder(results, :C, (-0.05, 3.5), "Cows Milking\n(Cows/Ha)", "(b)"),
    plotpowder(results, :P, (3,3500), "Pasture Cover\n(kg/Ha)","(c)"),
    plotpowder(results, :W, (0,150), "Soil Moisture\n(mm)","(d)"),
    plotpowder(results, :ev, (0,45), "Evapotranspiration\n(mm/Week)", "(e)"),
    plotpowder(results, :gr, (0,70), "Pasture Growth\n(kg/Day)", "(f)"),
    plotpowder(results, :fₛ, (0,110), "Supplement Fed\n(kg/Week)", "(g)"),
    plotpowder(results, :mlk, (0,4000), "Energy for Milk\n(MJ/Week)", "(h)"),
    layout=(4,2), size=(1000,1200)
)
savefig("farm.pdf")

In [351]:
using DataFrames, StatPlots
df = readtable("data/TGA.daily.df.csv")
q = [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
rainfall = unstack(by(df, :week) do io
   DataFrame(
        rainfall = quantile(io[:rainfall], q),
        quantile = q
        ) 
        end, :quantile, :rainfall)

evapotranspiration = unstack(by(df, :week) do io
   DataFrame(
        evapotranspiration = quantile(io[:evapotranspiration], q),
        quantile = q
        ) 
        end, :quantile, :evapotranspiration)

plot(rainfall, Symbol(0.5), w=3, c="#00467F")
plot!(rainfall, Symbol(0.0), fill=(Symbol(1.0), "#00467F"), fillalpha=0.25, alpha=0)
plot!(rainfall, Symbol(0.1), fill=(Symbol(0.9), "#00467F"), fillalpha=0.25, alpha=0)
plot!(rainfall, Symbol(0.25), fill=(Symbol(0.75), "#00467F"), fillalpha=0.25, alpha=0)
rainfall_plot = plot!(legend=false, title="(a)", xlabel="Week of Year", ylabel="Rainfall\n(mm/Week)")

plot(evapotranspiration, Symbol(0.5), w=3, c="#00467F")
plot!(evapotranspiration, Symbol(0.0), fill=(Symbol(1.0), "#00467F"), fillalpha=0.25, alpha=0)
plot!(evapotranspiration, Symbol(0.1), fill=(Symbol(0.9), "#00467F"), fillalpha=0.25, alpha=0)
plot!(evapotranspiration, Symbol(0.25), fill=(Symbol(0.75), "#00467F"), fillalpha=0.25, alpha=0)
evapotranspiration_plot = plot!(legend=false, title="(b)", xlabel="Week of Year", ylabel="Evapotranspiration\n(mm/Week)")

plot(rainfall_plot, evapotranspiration_plot, layout=(1,2), size=(1000,375), left_margin=8mm, bottom_margin=5mm)
savefig("weather.pdf")

In [471]:
using Query
y14_15 = @from i in df begin
    @where (i.year == 2014 && i.week >= 31) || (i.year == 2015 && i.week < 31)
    @select {i.evapotranspiration}
    @collect DataFrame
end
k14_15 = (15-2.9) * 1000 / sum(y14_15[:evapotranspiration])

y13_14 = @from i in df begin
    @where (i.year == 2013 && i.week >= 31) || (i.year == 2014 && i.week < 31)
    @select {i.evapotranspiration}
    @collect DataFrame
end
k13_14 = (15.9 - 2.8) * 1000 / sum(y13_14[:evapotranspiration])
k13_14, k14_15

(13.705796191671903, 13.664596273291925)

In [457]:
hgh = 2
plot(xlabel="Quantity (kg/Cow/Day)", ylabel="Price Multiplier",legend=false,ylims=(0,2hgh))
xbx = [0.0, 2.0, 2.0, 0.0]
ybx = [0, 0, 2hgh, 2hgh]
plot!(Plots.Shape(xbx, ybx), fillalpha=0.5, w=0, c="green", alpha=0)
plot!(Plots.Shape(2+xbx, ybx), fillalpha=0.5, w=0, c="orange", alpha=0)
plot!(Plots.Shape(4+xbx, ybx), fillalpha=0.5, w=0, c="red", alpha=0)
# plot!(Plots.Shape(6+xbx, ybx), fillalpha=0.6, w=0, c="red", alpha=0)
plot!([0, 2, 4, 6], 1+[0, 0, 1, 3], w=5, c="#00467F", size=(4 * 150,3 * 150), left_margin=5mm, bottom_margin=5mm)
annotate!([(1,hgh,text("FEI Grade A")), (3,hgh,text("FEI Grade B")), (5,hgh,text("FEI Grade C"))])
savefig("fei_penalty.pdf")